#### Topics:

    Stored Procedures

SQL Server stored procedures are used to group one or more Transact-SQL statements into logical units. The stored procedure is stored as a named object in the SQL Server Database Server.

When you call a stored procedure for the first time, SQL Server creates an execution plan and stores it in the cache. In the subsequent executions of the stored procedure, SQL Server reuses the plan to execute the stored procedure very fast with reliable performance.

In [1]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
#[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=BikeStores;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Create a cursor
cursor = conn.cursor()

#### 1. Creating simple stored Proc

In [2]:
cursor.execute('''
SELECT 
	product_name, 
	list_price
FROM 
	production.products
ORDER BY 
	product_name;
''')

#To create a stored procedure that wraps this query, you use the CREATE PROCEDURE statement as follows:

cursor.execute('''
CREATE PROCEDURE uspProductList
AS
BEGIN
    SELECT 
        product_name, 
        list_price
    FROM 
        production.products
    ORDER BY 
        product_name;
END;
''')



In this syntax:

    The uspProductList is the name of the stored procedure.
    The AS keyword separates the heading and the body of the stored procedure.
    If the stored procedure has one statement, the BEGIN and END keywords surrounding the statement are optional. 

You can find the stored procedure in the **Object Explorer, under Programmability > Stored Procedures**

#### 2. Executing simple stored proc


To execute a stored procedure, you use the EXECUTE or EXEC statement followed by the name of the stored procedure:

In [4]:
cursor.execute('''

EXECUTE uspProductList;

''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,899.99
1,Electra Amsterdam Fashion 7i Ladies' - 2017,1099.99
2,Electra Amsterdam Original 3i - 2015/2017,659.99
3,Electra Amsterdam Original 3i Ladies' - 2017,659.99
4,Electra Amsterdam Royal 8i - 2017/2018,1259.90
5,Electra Amsterdam Royal 8i Ladies - 2018,1199.99
6,Electra Cruiser 1 (24-Inch) - 2016,269.99
7,Electra Cruiser 1 (24-Inch) - 2016,269.99
8,Electra Cruiser 1 - 2016/2017/2018,269.99
9,Electra Cruiser 1 Ladies' - 2018,269.99


#### 3. Modifying a stored procedure

In [5]:
cursor.execute('''
 ALTER PROCEDURE uspProductList
    AS
    BEGIN
        SELECT 
            product_name, 
            list_price
        FROM 
            production.products
        ORDER BY 
            list_price 
    END;
''')

cursor.execute('''
EXEC uspProductList;
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Strider Classic 12 Balance Bike - 2018,89.99
1,Sun Bicycles Lil Kitt'n - 2017,109.99
2,Trek Boy's Kickster - 2015/2017,149.99
3,Trek Girl's Kickster - 2017,149.99
4,Trek Kickster - 2018,159.99
5,Trek Precaliber 12 Boys - 2017,189.99
6,Trek Precaliber 12 Girls - 2017,189.99
7,Trek Precaliber 12 Boy's - 2018,199.99
8,Trek Precaliber 12 Girl's - 2018,199.99
9,Trek Precaliber 16 Boy's - 2018,209.99


#### 4. Creating a stored procedure with one parameter

In [6]:
cursor.execute('''
SELECT
    product_name,
    list_price
FROM 
    production.products
ORDER BY
    list_price;
''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Strider Classic 12 Balance Bike - 2018,89.99
1,Sun Bicycles Lil Kitt'n - 2017,109.99
2,Trek Boy's Kickster - 2015/2017,149.99
3,Trek Girl's Kickster - 2017,149.99
4,Trek Kickster - 2018,159.99
5,Trek Precaliber 12 Boys - 2017,189.99
6,Trek Precaliber 12 Girls - 2017,189.99
7,Trek Precaliber 12 Boy's - 2018,199.99
8,Trek Precaliber 12 Girl's - 2018,199.99
9,Trek Precaliber 16 Boy's - 2018,209.99


In [7]:
cursor.execute('''
CREATE PROCEDURE uspFindProducts
AS
BEGIN
    SELECT
        product_name,
        list_price
    FROM 
        production.products
    ORDER BY
        list_price;
END;

''')

In [8]:
cursor.execute('''
ALTER PROCEDURE uspFindProducts(@min_list_price AS DECIMAL)
AS
BEGIN
    SELECT
        product_name,
        list_price
    FROM 
        production.products
    WHERE
        list_price >= @min_list_price
    ORDER BY
        list_price;
END;
''')

In this example:

    First, we added a parameter named @min_list_price to the uspFindProducts stored procedure. Every parameter must start with the @ sign. The AS DECIMAL keywords specify the data type of the @min_list_price parameter. The parameter must be surrounded by the opening and closing brackets.
    
    Second, we used @min_list_price parameter in the WHERE clause of the SELECT statement to filter only the products whose list prices are greater than or equal to the @min_list_price.

In [10]:
cursor.execute('''
EXEC uspFindProducts 500;
''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Electra Moto 1 - 2016,529.99
1,Electra Cruiser Lux 3i - 2018,529.99
2,Electra Cruiser Lux 3i Ladies' - 2018,529.99
3,Sun Bicycles Streamway 7 - 2017,533.99
4,Haro SR 1.1 - 2017,539.99
5,Haro Flightline Two 26 Plus - 2017,549.99
6,Electra Townie Original 21D - 2016,549.99
7,Electra Townie Original 21D - 2016,549.99
8,Sun Bicycles Streamway 3 - 2017,551.99
9,Electra Townie Original 21D - 2018,559.99


#### 5. Creating a stored procedure with multiple parameters

Stored procedures can take one or more parameters. The parameters are separated by commas.

The following statement modifies the uspFindProducts stored procedure by adding one more parameter named @max_list_price to it:

In [12]:
cursor.execute('''
ALTER PROCEDURE uspFindProducts(
    @min_list_price AS DECIMAL
    ,@max_list_price AS DECIMAL
)
AS
BEGIN
    SELECT
        product_name,
        list_price
    FROM 
        production.products
    WHERE
        list_price >= @min_list_price AND
        list_price <= @max_list_price
    ORDER BY
        list_price;
END;

''')

Once the stored procedure is modified successfully, you can execute it by passing two arguments, one for @min_list_price and the other for @max_list_price:

In [14]:
cursor.execute('''
EXECUTE uspFindProducts 900, 1000;
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Electra Straight 8 3i - 2018,909.99
1,Trek X-Caliber 7 - 2018,919.99
2,Trek Stache Carbon Frameset - 2018,919.99
3,Trek Domane AL 3 - 2018,919.99
4,Trek Domane AL 3 Women's - 2018,919.99
5,Trek CrossRip 1 - 2018,959.99
6,Electra Delivery 3i - 2016/2017/2018,959.99
7,Surly Wednesday Frameset - 2016,999.99
8,Surly Big Dummy Frameset - 2017,999.99
9,Trek X-Caliber 8 - 2017,999.99


In [15]:
#Using named parameters

cursor.execute('''
EXECUTE uspFindProducts 
    @min_list_price = 900, 
    @max_list_price = 1000;

''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Electra Straight 8 3i - 2018,909.99
1,Trek X-Caliber 7 - 2018,919.99
2,Trek Stache Carbon Frameset - 2018,919.99
3,Trek Domane AL 3 - 2018,919.99
4,Trek Domane AL 3 Women's - 2018,919.99
5,Trek CrossRip 1 - 2018,959.99
6,Electra Delivery 3i - 2016/2017/2018,959.99
7,Surly Wednesday Frameset - 2016,999.99
8,Surly Big Dummy Frameset - 2017,999.99
9,Trek X-Caliber 8 - 2017,999.99


In [17]:
# Creating text parameters

cursor.execute('''
ALTER PROCEDURE uspFindProducts(
    @min_list_price AS DECIMAL,
    @max_list_price AS DECIMAL,
    @name AS VARCHAR(max)
)
AS
BEGIN
    SELECT
        product_name,
        list_price
    FROM 
        production.products
    WHERE
        list_price >= @min_list_price AND
        list_price <= @max_list_price AND
        product_name LIKE '%' + @name + '%'
    ORDER BY
        list_price;
END;
''')

'''
In the WHERE clause of the SELECT statement, we added the following condition:

product_name LIKE '%' + @name + '%'
Code language: SQL (Structured Query Language) (sql)
By doing this, the stored procedure returns the products whose list prices are in the range of min and max list prices 
and the product names also contain a piece of text that you pass in.
'''
cursor.execute('''
EXECUTE uspFindProducts 
    @min_list_price = 900, 
    @max_list_price = 1000,
    @name = 'Trek';

''')
# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_name,list_price
0,Trek X-Caliber 7 - 2018,919.99
1,Trek Stache Carbon Frameset - 2018,919.99
2,Trek Domane AL 3 - 2018,919.99
3,Trek Domane AL 3 Women's - 2018,919.99
4,Trek CrossRip 1 - 2018,959.99
5,Trek X-Caliber 8 - 2017,999.99
6,Trek X-Caliber 8 - 2018,999.99
7,Trek Farley Carbon Frameset - 2018,999.99


In [18]:
conn.close()
conn.closed

True